In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_baatcheet, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "fnnerd/Baatcheet-7b", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

adapter_config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/200M [00:00<?, ?B/s]

Unsloth 2024.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
prompt = "Translation ### English:'You don't know the movie Sholey?? Such an iconic movie! And I love the character Gabbar. How nicely crafted.' to ### Hinglish: "

inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
outputs = model_baatcheet.generate(**inputs, max_new_tokens = 64)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Translation ### English:'You don't know the movie Sholey?? Such an iconic movie! And I love the character Gabbar. How nicely crafted.' to ### Hinglish: <em>'Sholey movie ko tum nahi jaante?? Itni iconic movie hai! Aur main Gabbar character ko bahut pasand karta hoon. Kitna achha bana hai!'</em>


In [11]:
prompt = "Translation ### English:''Which character from 'Lord of the Rings' is your favorite?'' to ### Hinglish: "

inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
outputs = model_baatcheet.generate(**inputs, max_new_tokens = 64)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Translation ### English:''Which character from 'Lord of the Rings' is your favorite?'' to ### Hinglish: <em>'Lord of the Rings' se apna favorite character kaunsa hai?</em>
